# DingoDB | DingoDB> [DingoDB](https://dingodb.readthedocs.io/en/latest/)は、データレイクとベクターデータベースの特性を組み合わせた分散型マルチモードベクターデータベースであり、あらゆるタイプやサイズのデータ（キーバリュー、PDF、オーディオ、ビデオなど）を保存することができます。リアルタイムで低遅延の処理能力を備え、迅速な洞察と対応を実現し、マルチモーダルデータの即時分析と処理を効率的に行うことができます。>> > [DingoDB](https://dingodb.readthedocs.io/en/latest/) is a distributed multi-mode vector database, which combines the characteristics of data lakes and vector databases, and can store data of any type and size (Key-Value, PDF, audio, video, etc.). It has real-time low-latency processing capabilities to achieve rapid insight and response, and can efficiently conduct instant analysis and process multi-modal data.このノートブックは、DingoDBベクターデータベースに関連する機能の使用方法を示しています。> This notebook shows how to use functionality related to the DingoDB vector database.実行するには、[DingoDBインスタンスが起動して動作している](https://github.com/dingodb/dingo-deploy/blob/main/README.md)状態である必要があります。> To run, you should have a [DingoDB instance up and running](https://github.com/dingodb/dingo-deploy/blob/main/README.md).

In [ ]:
!pip install dingodb
# or install latest:
!pip install git+https://git@github.com/dingodb/pydingo.git

OpenAIEmbeddingsを使用するためには、OpenAI APIキーを取得する必要があります。> We want to use OpenAIEmbeddings so we have to get the OpenAI API Key.

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:········


In [2]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Dingo

In [3]:
from langchain.document_loaders import TextLoader

loader = TextLoader("../../modules/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [4]:
from dingodb import DingoDB

index_name = "langchain_demo"

dingo_client = DingoDB(user="", password="", host=["127.0.0.1:13000"])
# First, check if our index already exists. If it doesn't, we create it
if (
    index_name not in dingo_client.get_index()
    and index_name.upper() not in dingo_client.get_index()
):
    # we create a new index, modify to your own
    dingo_client.create_index(
        index_name=index_name, dimension=1536, metric_type="cosine", auto_id=False
    )

# The OpenAI embedding model `text-embedding-ada-002 uses 1536 dimensions`
docsearch = Dingo.from_documents(
    docs, embeddings, client=dingo_client, index_name=index_name
)

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Dingo

In [5]:
query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query)

In [2]:
print(docs[0].page_content)

### Adding More Text to an Existing Index | 既存のインデックスに更にテキストを追加する`add_texts` 関数を使用して、既存のDingoインデックスにテキストを追加または挿入することができます> More text can embedded and upserted to an existing Dingo index using the `add_texts` function

In [ ]:
vectorstore = Dingo(embeddings, "text", client=dingo_client, index_name=index_name)

vectorstore.add_texts(["More text!"])

### Maximal Marginal Relevance Searches | 最大限余剰関連性検索リトリバーオブジェクトで類似性検索を使用することに加えて、`mmr`をリトリバーとしても使用できます。> In addition to using similarity search in the retriever object, you can also use `mmr` as retriever.

In [ ]:
retriever = docsearch.as_retriever(search_type="mmr")
matched_docs = retriever.get_relevant_documents(query)
for i, d in enumerate(matched_docs):
    print(f"\n## Document {i}\n")
    print(d.page_content)

`max_marginal_relevance_search`を直接使用することもできます：> Or use `max_marginal_relevance_search` directly:

In [ ]:
found_docs = docsearch.max_marginal_relevance_search(query, k=2, fetch_k=10)
for i, doc in enumerate(found_docs):
    print(f"{i + 1}.", doc.page_content, "\n")